In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

cnn = Net()
cnn.load_state_dict(torch.load('mnist_cnn.pt'))

<All keys matched successfully>

In [13]:
import toml
import json
import yaml

sd = dict(cnn.state_dict())
mds = list(cnn.modules())
mds[1:][0].__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_pre_hooks', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_hooks_always_called', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_state_dict_hooks', '_state_dict_pre_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'in_channels', 'out_channels', 'kernel_size', 'stride', 'padding', 'dilation', 'transposed', 'output_padding', 'groups', 'padding_mode', '_reversed_padding_repeated_twice'])

In [40]:
dict(cnn.conv1.__dict__)

d = { k:v for (k,v) in dict(cnn.conv1.__dict__).items() if k[0] != '_'}


In [43]:
# print(json.dumps(d,indent=2))
print(toml.dumps(d))

training = true
in_channels = 1
out_channels = 32
kernel_size = [ 3, 3,]
stride = [ 1, 1,]
padding = [ 0, 0,]
dilation = [ 1, 1,]
transposed = false
output_padding = [ 0, 0,]
groups = 1
padding_mode = "zeros"



In [76]:
def get_attributes(model):
    return { str(k):v for (k,v) in dict(model.__dict__).items() if k[0] != '_'}

def get_summary(model,global_name,parent_name=None):
    model_name = model.__class__.__name__
    d = {
        'layer': model_name,
        'attributes': get_attributes(model),
        'sub_modules': { name : get_summary(m,global_name=global_name,parent_name=name) for name,m in  model.named_children() }
    }
    if parent_name is None:
        return {
            'model_name': global_name,
            global_name: d
        }
    return d
# get_summary(cnn)
print(toml.dumps(get_summary(cnn,'cnn')))
# print(yaml.dump(get_summary(cnn)))
# print(json.dumps(get_summary(cnn),indent=2))

model_name = "cnn"

[cnn]
layer = "Net"

[cnn.attributes]
training = true

[cnn.sub_modules.conv1]
layer = "Conv2d"

[cnn.sub_modules.conv2]
layer = "Conv2d"

[cnn.sub_modules.dropout1]
layer = "Dropout"

[cnn.sub_modules.dropout2]
layer = "Dropout"

[cnn.sub_modules.fc1]
layer = "Linear"

[cnn.sub_modules.fc2]
layer = "Linear"

[cnn.sub_modules.conv1.attributes]
training = true
in_channels = 1
out_channels = 32
kernel_size = [ 3, 3,]
stride = [ 1, 1,]
padding = [ 0, 0,]
dilation = [ 1, 1,]
transposed = false
output_padding = [ 0, 0,]
groups = 1
padding_mode = "zeros"

[cnn.sub_modules.conv1.sub_modules]

[cnn.sub_modules.conv2.attributes]
training = true
in_channels = 32
out_channels = 64
kernel_size = [ 3, 3,]
stride = [ 1, 1,]
padding = [ 0, 0,]
dilation = [ 1, 1,]
transposed = false
output_padding = [ 0, 0,]
groups = 1
padding_mode = "zeros"

[cnn.sub_modules.conv2.sub_modules]

[cnn.sub_modules.dropout1.attributes]
training = true
p = 0.25
inplace = false

[cnn.sub_modules.dropout

In [78]:
with open('../data/models/mnist_cnn/summary.toml','w') as f:
    f.write(toml.dumps(get_summary(cnn,'cnn')))
with open('../data/models/mnist_cnn/summary.json','w') as f:
    f.write(json.dumps(get_summary(cnn,'cnn'),indent=2))